In [186]:
import pandas as pd
import re
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from pprint import pprint
import pyLDAvis.gensim
import pickle
import pyLDAvis
import docx2txt
import os
import nltk

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [145]:
os.chdir("C:\\Users\\vanandkuma\\Documents\\projects\\sentiment_analysis")

In [146]:
os.chdir('input/app_v1')

path_v1 = ("C:\\Users\\vanandkuma\\Documents\\projects\\sentiment_analysis\\input\\app_v1")

path_v2 = ("C:\\Users\\vanandkuma\\Documents\\projects\\sentiment_analysis\\input\\app_v2")

In [147]:
files = []
for file in os.listdir(path_v1):
    if file.endswith('.docx'):
        files.append(file)

In [148]:
pooled_doc = []
for i in range(len(files)):
    text = docx2txt.process(files[i])
    pooled_doc.append(text) 

In [149]:
len(pooled_doc)

4

In [150]:
df = []
for i in range(len(pooled_doc)):
    sentence = nltk.tokenize.sent_tokenize(pooled_doc[i])
    for i in range(len(sentence)):
        df.append(sentence[i])

In [151]:
for i in range(len(df)):
    df[i] = df[i].replace('\n', '')

In [153]:
df = pd.DataFrame(df, columns=['text'])

In [154]:
df['text_processed'] = \
df['text'].map(lambda x: re.sub('[,.!?]', '', x))

In [155]:
df['text_processed'] = \
df['text_processed'].map(lambda x: x.lower())

In [156]:
df.head()

,text,text_processed
0,"Startseite OK.Komm, wir gehen Baby.",startseite okkomm wir gehen baby
1,"Unterstützung beim Ärztepflegende, Schwangere ...",unterstützung beim ärztepflegende schwangere u...
2,Schöne und sichere Gebot füralle Beteiligten O...,schöne und sichere gebot füralle beteiligten o...
3,Aber jetzt starten kommt danndie Registration ...,aber jetzt starten kommt danndie registration ...
4,Nur so ist jetzt eine ganzkomische kleine Sach...,nur so ist jetzt eine ganzkomische kleine sach...


In [168]:
stop_words = stopwords.words('german')
stop_words.extend(['ja', 'ne', 'ok', 'okay', 'genau', 'also', 'mhm'])

In [169]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), 
                                             deacc = True))

In [170]:
def remove_stopwords(texts):
    return[[word for word in simple_preprocess(str(doc)) 
            if word not in stop_words] for doc in texts]

In [171]:
data = df.text_processed.values.tolist()
data_words = list(sent_to_words(data))

In [172]:
data_words = remove_stopwords(data_words)

In [ ]:
#print(data_words)

In [173]:
id2word = corpora.Dictionary(data_words)

In [174]:
texts = data_words

In [175]:
corpus = [id2word.doc2bow(text) for text in texts]

In [176]:
num_topics = 2

In [177]:
lda_model = gensim.models.LdaMulticore(corpus = corpus, 
                                       id2word = id2word, 
                                       num_topics = num_topics)

In [188]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.7072971706798725

Coherence Score:  0.5932079125791875


C:\Users\vanandkuma\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\_memmapping_reducer.py:611: UserWarning: Failed to delete temporary folder: C:\Users\VANAND~1\AppData\Local\Temp\joblib_memmapping_folder_83356_cf8b2b48ab284b5eb1cdbf9e208dc622_24d9f9dd0c00418aab282ecaa889abbd
  warnings.warn("Failed to delete temporary folder: {}"
C:\Users\vanandkuma\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\_memmapping_reducer.py:611: UserWarning: Failed to delete temporary folder: C:\Users\VANAND~1\AppData\Local\Temp\joblib_memmapping_folder_83356_926db34164c14416b3c7e2915cca10d8_2f8d8b9b70ca4c50b96b469f99ccec60
  warnings.warn("Failed to delete temporary folder: {}"
C:\Users\vanandkuma\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\_memmapping_reducer.py:611: UserWarning: Failed to delete temporary folder: C:\Users\VANAND~1\AppData\Local\Temp\joblib_memmapping_folder_83356_cf8b2b48ab284b5eb1cdbf9e208dc622_6d8c2d5fdd834e0194f715b2acb7713c
  w

In [178]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"gut" + 0.010*"mal" + 0.008*"schon" + 0.008*"fur" + 0.007*"irgendwie" '
  '+ 0.007*"ganz" + 0.007*"einfach" + 0.006*"super" + 0.006*"konnen" + '
  '0.006*"immer"'),
 (1,
  '0.018*"schon" + 0.015*"mal" + 0.013*"gut" + 0.010*"einfach" + 0.008*"immer" '
  '+ 0.007*"vielleicht" + 0.007*"geburt" + 0.006*"bisschen" + 0.006*"nochmal" '
  '+ 0.006*"halt"')]


In [179]:
pyLDAvis.enable_notebook()

LDAvis_data_filepath = 'output/ldavis_prepared_topics_three'

In [181]:
os.chdir("C:\\Users\\vanandkuma\\Documents\\projects\\sentiment_analysis")

In [182]:
if __name__ == '__main__':
    if 1 == 1:
        LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, 
                                                  id2word)
        with open(LDAvis_data_filepath, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)

In [183]:
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [184]:
LDAvis_prepared

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.018005  0.0       1        1  51.727773
0     -0.018005  0.0       2        1  48.272227, topic_info=              Term        Freq       Total Category  logprob  loglift
135          schon  285.000000  285.000000  Default  30.0000  30.0000
263        wichtig   48.000000   48.000000  Default  29.0000  29.0000
74            hast   54.000000   54.000000  Default  28.0000  28.0000
799           gibt   80.000000   80.000000  Default  27.0000  27.0000
41           super  100.000000  100.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
275  kommunikation   35.472469   76.002940   Topic2  -5.6744  -0.0337
140          wurde   37.338939   86.885442   Topic2  -5.6231  -0.1162
35        bisschen   40.907578  112.008564   Topic2  -5.5318  -0.2789
220         geburt   41.225119  115.364399   Topic2  -5.5241  -0.3007
587     eigentlich   35.909789   82.726066   Topic2  -5.6621  -0.1062

[190 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2016      1  0.837830   achten
2016      2  0.239380   achten
398       1  0.340987       ah
398       2  0.664027       ah
1991      1  0.245293   alters
...     ...       ...      ...
140       2  0.425848    wurde
649       1  0.756282   wusste
649       2  0.189070   wusste
1374      1  0.717820  zuhause
1374      2  0.239273  zuhause

[273 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])